 <div class="alert alert-block alert-info">
    <b>Note:</b> 
    Interchange is in the process of replacing ParmEd in many workflows, but it still in an alpha testing phase. Our internal tests indicate it is reliable for many small-molecule systems, but it is not yet reliable for complex, multi-component systems and there are likely still rough edges throughout. Feedback is welcome on the <a href=https://github.com/openforcefield/openff-interchange/issues/>Interchange issue tracker.</a></div>

## Using OpenFF force fields in Amber and GROMACS

The Open Forcefield Toolkit can create parametrized `openmm.System` objects that can be natively simulated with OpenMM. This example shows the Interchange project can enable parallel workflows using Amber and GROMACS.

### Preparing an OpenFF Topology

We start by loading a PDB file containing one copy of ethanol and cyclohexane. Our goal is to create an OpenFF `Topology` object describing this system that we can parametrize with the SMIRNOFF-format "Sage" force field.

The two `Molecule` objects created from the SMILES strings can contain information such as formal charges and stereochemistry that is not included in a PDB file. These objects are passed to the PDB loading function via the `unique_molecules` argument. In this example, partial charges are not explicitly given, and `ForceField` will assign AM1/BCC charges as specified by the "Sage" force field. Note that the OpenFF Toolkit produces partial charges that do not depend on the input conformation of parameterized molecules. See the [FAQ](https://open-forcefield-toolkit.readthedocs.io/en/latest/faq.html#the-partial-charges-generated-by-the-toolkit-don-t-seem-to-depend-on-the-molecule-s-conformation-is-this-a-bug) for more information.

In [1]:
from openff.toolkit import ForceField, Molecule, Topology
from openff.units import Quantity, unit
from openmm import unit as openmm_unit

LICENSE: Could not open license file specified by OE_LICENSE environment variable "/home/USERNAME/oe_license.txt"
LICENSE: Could not open license file "oe_license.txt" in local directory
LICENSE: N.B. OE_DIR environment variable is not set
LICENSE: No product keys!
LICENSE: No product keys!
LICENSE: No product keys!
The OpenEye Toolkits are found to be installed but not licensed and therefore will not be used.
The OpenEye Toolkits require a (free for academics) license, see https://docs.eyesopen.com/toolkits/python/quickstart-python/license.html
LICENSE: No product keys!


In [2]:

from rdkit import Chem
mol = Chem.MolFromPDBFile('file.pdb', removeHs=False)
Chem.MolToPDBFile(mol, 'file_CONECT.pdb')
from openff.toolkit import Molecule, Topology
TBP = Molecule.from_smiles("O=P(OCCCC)(OCCCC)OCCCC")
topology = Topology.from_pdb( "file_CONECT.pdb", unique_molecules=[TBP] )


topology

In [3]:
topology.box_vectors = [30, 30, 30] * unit.angstrom
nonbondedCutoff=15* unit.angstrom

### Preparing an OpenFF ForceField

Once the `ForceField` class is imported, the only decision to make is which force field to use. An exhaustive list of force fields released by the Open Force Field Initiative can be found [here](https://github.com/openforcefield/openff-forcefields#how-to-cite).

Here we will use force field from the "Sage" line.

In [1]:
forcefield = ForceField("openff-2.1.0.offxml")
forcefield

NameError: name 'ForceField' is not defined

### Preparing an OpenMM System

Once a force field and topology have been loaded, an `openmm.System` can be generated natively with the OpenFF Toolkit.

In [8]:
omm_system = forcefield.create_openmm_system(topology)
omm_system

<openmm.openmm.System; proxy of <Swig Object of type 'OpenMM::System *' at 0x1529bdc20> >

### Preparing an Interchange object

To exports to engines other than OpenMM, we will make use of the [Interchange](https://openff-interchange.readthedocs.io/) project. There is a high-level `Interchange.from_smirnoff` function that consumes OpenFF Toolkit and ForceField objects and produces an `Interchange` object which can then be exported to formats understood by other molecular simulation engines. This extra step is needed to provide a clean interface between _applied_ parameters and engines. Note also that this step does not require an OpenMM System to be generated; `ForceField.create_openmm_system` does not need to be called to use Amber and GROMACS.

In [9]:
from openff.interchange import Interchange

interchange = Interchange.from_smirnoff(
    force_field=forcefield,
    topology=topology,
)
interchange

Interchange with 7 collections, periodic topology with 44 atoms.

### Exporting to Amber and GROMACS files

Once an `Interchange` object has been constructed, its API can be used to export to files understood by GROMACS, Amber, and more.

In [10]:
# Export AMBER files.
interchange.to_prmtop("system.prmtop")
interchange.to_inpcrd("system.inpcrd")

# Export GROMACS files.
interchange.to_top("system.top")
interchange.to_gro("system.gro")

### Validating the conversion to Amber files

The Interchange project includes functions that take in an `Interchange` object and call out to simulation engines to run single-point energy calculations (with no minimization or dynamics) for the purpose of validating the export layer with each engine. Under the hood, each of these functions calls API points like those used above while converting to files understood by each engine. These rely on having each engine installed and accessible in the current `$PATH`.

In [11]:
from openff.interchange.drivers import get_amber_energies, get_openmm_energies

In [12]:
openmm_energies = get_openmm_energies(interchange)
openmm_energies.energies

{'Bond': 3.856400101835694 <Unit('kilojoule / mole')>,
 'Angle': 42.267547941828205 <Unit('kilojoule / mole')>,
 'Torsion': 300.3869711608175 <Unit('kilojoule / mole')>,
 'Nonbonded': -316.3951029849092 <Unit('kilojoule / mole')>}

In [ ]:
amber_energies = get_amber_energies(interchange)
amber_energies.energies

### Appendix: Validating the conversion to GROMACS and LAMMPS files

If GROMACS and/or LAMMPS are installed on your machine, the same comparisons can also take place with those engines. They are available via `conda` by running a command like:

```conda install "gromacs >=2021=nompi*" lammps -c conda-forge```

In [20]:
from distutils.spawn import find_executable
from pprint import pprint

from openff.interchange.drivers import get_gromacs_energies, get_lammps_energies

In [21]:
if find_executable("lmp_serial"):
    pprint(get_lammps_energies(interchange).energies)

{'Angle': <Quantity(10.102186, 'kilocalorie_per_mole')>,
 'Bond': <Quantity(1.2340399, 'kilocalorie_per_mole')>,
 'Electrostatics': <Quantity(-75.60622, 'kilocalorie_per_mole')>,
 'Torsion': <Quantity(71.794209, 'kilocalorie_per_mole')>,
 'vdW': <Quantity(-0.0319007708, 'kilocalorie_per_mole')>}


In [22]:
if find_executable("gmx"):
    pprint(get_gromacs_energies(interchange).energies)

{'Angle': <Quantity(42.2679138, 'kilojoule / mole')>,
 'Bond': <Quantity(3.85664272, 'kilojoule / mole')>,
 'Electrostatics': <Quantity(-316.597072, 'kilojoule / mole')>,
 'RBTorsion': <Quantity(0.0, 'kilojoule / mole')>,
 'Torsion': <Quantity(300.387054, 'kilojoule / mole')>,
 'vdW': <Quantity(0.0683389157, 'kilojoule / mole')>}


Finally, there is a helper function `get_summary_data` that will attempt to run drivers of each engine. A summary reported is prepared as a Pandas `DataFrame`.

In [23]:
from openff.interchange.drivers.all import get_summary_data

get_summary_data(interchange)

,Bond,Angle,Torsion,Electrostatics,vdW,RBTorsion
OpenMM,3.856400,42.267548,300.386971,-316.436456,0.041356,NaN
Amber,3.856393,42.267605,300.386933,-316.424205,1.200808,NaN
GROMACS,3.856643,42.267914,300.387054,-316.597072,0.068339,0.0
LAMMPS,5.163223,42.267546,300.386970,-316.336424,-0.133473,NaN
